In [1]:
import numpy as np
from symmer.symplectic import (AntiCommutingOp, PauliwordOp, 
                               random_anitcomm_2n_1_PauliwordOp)

from symmer.symplectic.anticommuting_op import conjugate_Pop_with_R

# 1. Given an anticommuting set of operators

In [39]:
# AC_dict = {'ZZZ':2, 'XXX':1, 'YYY':2}
# test_op = AntiCommutingOp.from_dictionary(AC_dict)


### randomly generate anticommuting operator
nq = 5
test_op = AntiCommutingOp.from_PauliwordOp(random_anitcomm_2n_1_PauliwordOp(nq))
test_op.coeff_vec = (test_op.coeff_vec.real).astype(complex)
test_op.n_terms

11

In [9]:
test_op

-0.162+0.000j YYXZY +
-0.292+0.000j ZYYYI +
-0.233+0.000j IXIII +
 0.179+0.000j YZYZX +
 1.328+0.000j IZYIZ +
-1.635+0.000j ZZIYY +
-0.292+0.000j YZXII +
-0.671+0.000j XZIZX +
 0.029+0.000j ZZIIX +
 2.092+0.000j XYZXY +
-0.875+0.000j XZXYI

# 2. Use unitary partitioning to find unitary mapping to a single term in the linear combination

For full details see the following papers:
- [PAPER](https://arxiv.org/abs/1908.08067)
- [PAPER](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.3.033195)

TWO methods:
1. linear combination of unitaries (LCU)
2. a sequence of (exponentiated pauli) rotations

## 2.1 LCU method

In [37]:
# s_index gives index in symplectic matrix of Pauli term to be reduced too
# if NOT defined then s_index is automatically chosen to be least Pauli dense term

Ps_LCU, rotations_LCU, gamma_l_LCU, normed_clique_LCU = test_op.unitary_partitioning(up_method='LCU',
                                                                                    s_index=None)

# Ps_LCU = term that is reduced too
# rotations_LCU = PauliwordOp of LCU unitary that does this
# gamma_l_LCU = normalization factor of clique
# normed_clique_LCU = normalized anticommuting operator

In [32]:
# clique normalized by gamma_l
test_op == normed_clique_LCU.multiply_by_constant(gamma_l_LCU)

True

In [33]:
# check reduction
rotations_LCU * normed_clique_LCU * rotations_LCU.conjugate

 1.000+0.000j IXIII

In [34]:
# other method to perform conjugation
conjugate_Pop_with_R(normed_clique_LCU, rotations_LCU)

 1.000+0.000j IXIII

In [18]:
# note what happens if rotation performed without normalization
print('normaliziation factor:', gamma_l_LCU)

rotations_LCU * test_op * rotations_LCU.conjugate

normaliziation factor: 3.2116805082177207


 3.212+0.000j IXIII

## 2.2 Sequence of Rotations method

In [38]:
# Ps_SeqRot = term that is reduced too
# rotations_SeqRot = list of PauliwordOp rotations. List of tuples of (P, angle_of_rotation)
# gamma_l_SeqRot = normalization factor of clique
# normed_clique_SeqRot = normalized anticommuting operator
(Ps_SeqRot, rotations_SeqRot, 
 gamma_l_SeqRot, normed_clique_SeqRot) = test_op.unitary_partitioning(up_method='seq_rot',
                                                                      s_index=None)

In [21]:
# clique normalized by gamma_l
test_op == normed_clique_SeqRot.multiply_by_constant(gamma_l_SeqRot)

True

In [22]:
# apply sequence of rotations on anticommuting operator (normalized)
normed_clique_SeqRot.perform_rotations(rotations_SeqRot)

 1.000+0.000j IXIII

In [23]:
# without normalization
test_op.perform_rotations(rotations_SeqRot)

 3.212+0.000j IXIII

# 3. Further work

Some methods require applying unitary partitioning operator (unitary of LCU or seq_rot) to other combinations of Pauli operators (such as a Hamiltonian)

For examples see:
- [PAPER1](https://arxiv.org/abs/2011.10027) 
- [PAPER2](https://arxiv.org/abs/2204.02150)
- [PAPER3](https://arxiv.org/abs/2207.03451)

This subsection looks at the scaling of the different methods

In [24]:
### Generate a random linear combination of Pauli operators (on same no. of qubits as rotations above)

n_terms= 30
# n_terms= 500
# n_terms= 12
op = PauliwordOp.random(test_op.n_qubits, n_terms)

# op = PauliwordOp.haar_random(test_op.n_qubits)
op.n_terms

30

In [26]:
## SeqRot scaling
print('max possible P terms on defined qubits:', 4**rotations_LCU.n_qubits)
print('max terms SeqRot:', 2**rotations_LCU.n_terms * op.n_terms)

max possible P terms on defined qubits: 1024
max terms SeqRot: 61440


In [27]:
op.perform_rotations(rotations_SeqRot).n_terms

755

In [28]:
## LCU scaling
print('max terms on qubits:', 4**rotations_LCU.n_qubits)
print('max terms LCU:', rotations_LCU.n_terms**2 * op.n_terms)

max terms on qubits: 1024
max terms LCU: 3630


In [29]:
# manual multiplication
(rotations_LCU * op * rotations_LCU.conjugate).n_terms

570

**Important**: note how LCU method results in **fewer** terms!

- SeqRot method increases terms as ~$2^{|U|}$
- LCU method increases terms as  ~$|U|^{2}$

where $|U|$ is the number of Pauli terms in the operator performing unitary partitioning.

for further details see [PAPER](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.3.033195)